# NBA Rebounding Evolution Dashboard

The purpose of this project is to practice my Dashboard skills by evaluating how rebounding has evolved in the NBA, to accompany the [medium](https://medium.com/@davidve0206/evolution-of-nba-rebounding-b1aef152c715) blog that explained my EDA regarding this subject.

The data used for the project comes from [this](https://www.kaggle.com/datasets/sumitrodatta/nba-aba-baa-stats) dataset on Kaggle; as the datasets are provided as csv files, I created a simple SQLite database with the two files I am most interest in, Player Per Game and Team Summaries, which has been placed in this repository after creation. Moreover, this project uses the Views created in the previous EDA notebook (also in this repository) as a starting point, so I would recommend checking that file before.

Finally, this file was created to work locally; the file that will be used for deployment is "nbarebound_dashboard.py"

Given that, we will being by importing all required packages for this lab:

In [1]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, dash_table, Input, Output

%load_ext sql
%sql sqlite:///nbadatabase.sqlite

'Connected: @nbadatabase.sqlite'

Now we will bring the view created in the EDA Lab as a DataFrame and create other useful variables:

In [2]:
rebounds_df = %sql SELECT player, season, team, player_type, trb_per_game FROM player_stats_rebounds
rebounds_df = rebounds_df.DataFrame()
rebounds_df['season'] = pd.to_numeric(rebounds_df['season'])
rebounds_df['trb_per_game'] = pd.to_numeric(rebounds_df['trb_per_game'])

seasons =  list(rebounds_df['season'].unique())
output_columns = ['Player', 'Average Rebounds per Game']
colors = {
    'nba_blue': '#17408B',
    'white': '#FFFFFF',
    'nba_red': '#C9082A',
    'light_blue': '#acc0e6'
}

 * sqlite:///nbadatabase.sqlite
Done.


The code below defines functions that will be used in the dashboard:

In [13]:
def get_rebounds_df(season, player_type):
    
    if player_type is not None:
        rebounds_bygame_df = rebounds_df[rebounds_df['player_type']==player_type]
    else:
        rebounds_bygame_df = rebounds_df
    
    if season is not None:
        rebounds_bygame_df = rebounds_bygame_df[rebounds_bygame_df['season']==season]
    
    rebounds_avg_df = rebounds_bygame_df[['player','trb_per_game']].groupby(['player']).mean(numeric_only=True).round(1).reset_index()
    rebounds_avg_df = rebounds_avg_df.sort_values('trb_per_game', ascending=False)
    rebounds_avg_df.columns = output_columns

    return rebounds_avg_df   

def get_rebounds_hist(rebounds_avg_df):
    fig = px.violin(rebounds_avg_df, x='Average Rebounds per Game', hover_data=['Player'], box=True)
    fig.update_layout(title="Distribution of Rebounds per Game", width=600, height=350)
    return fig

def get_rebounds_line(player_type):
    if player_type is not None:
        rebounds_evolution_df = rebounds_df[rebounds_df['player_type']==player_type]
    else:
        rebounds_evolution_df = rebounds_df
    rebounds_evolution_df = rebounds_evolution_df[['season','trb_per_game']].groupby('season').mean(numeric_only=True).reset_index()

    fig = px.line(rebounds_evolution_df, x='season', y='trb_per_game')
    fig.update_layout(title=f"Evolution of rebounds per game from {player_type if player_type is not None else 'all players'}",
                xaxis_title='Season',
                yaxis_title='Rebounds per Game',
                width=600, height=350)
    
    return fig

def get_rebounds_bar(season, player_type):
    if player_type is not None:
        rebounds_bygame_df = rebounds_df[(rebounds_df['season']==season) & (rebounds_df['player_type']==player_type)]
    else:
        rebounds_bygame_df = rebounds_df[rebounds_df['season']==season]
    rebounds_bygame_df = rebounds_bygame_df[['team','trb_per_game']].groupby('team').mean(numeric_only=True).reset_index()
    rebounds_bygame_df = rebounds_bygame_df.sort_values('trb_per_game', ascending=False).head(10)

    fig = px.bar(rebounds_bygame_df, x='team', y='trb_per_game')
    fig.update_layout(title=f"Top teams in rebounds per game from {player_type if player_type is not None else 'all players'}",
                xaxis_title='Team Abbreviation',
                yaxis_title='Rebounds per Game',
                width=600, height=350)
    
    return fig

The code below creates the structure of the dashboard and initiates it:

In [24]:
app = JupyterDash(__name__)

app.layout = html.Div(style={'backgroundColor': colors['nba_blue']},
                      children=[
                                html.H1(children='NBA Rebounding Dashboard',
                                        style={
                                            'textAlign': 'center',
                                            'color': colors['white']
                                        }),
                                
                                # Outer Division for dropdowns
                                html.Div([
                                        #Dropdown helper 1: Year
                                        html.Div([html.H3('Season:', style={'margin-right': '1em', 'color': colors['white']})]),
                                        #Dropdown 1: Year
                                        dcc.Dropdown(id='input-season',
                                             options=[{'label': i, 'value': i} for i in seasons],
                                             placeholder='Select a year',
                                             style={'width':'40%', 'padding':'1px', 'font-size': '15px', 'text-align': 'center'}
                                             ),
                                        #Dropdown helper 2: Player Type
                                        html.Div([html.H3('Player Type:', style={'margin-right': '1em', 'color': colors['white']})]),
                                        #Dropdown 2: Player Type
                                        dcc.Dropdown(id='input-ptype',
                                             options=[{'label': 'Big', 'value': 'Big'},
                                                      {'label': 'Guard', 'value': 'Guard'}],
                                             placeholder='Select a player type',
                                             style={'width':'40%', 'padding':'1px', 'font-size': '15px', 'text-align': 'center'}
                                             )
                                        ], style = {'display': 'flex', 'align-items': 'center', 'justify-content': 'center', 'margin-left': '10em'}),

                                # Outer Division for graphs
                                html.Div([
                                        html.Div(
                                                html.Table([
                                                            html.Tr('Top 10 Rebounders', style={'font-size': '2.0rem', 'textAlign': 'center', 'color': colors['white']}),
                                                            html.Tr(id='top10-table')])
                                                , style={'margin': '5px'}),
                                        
                                                html.Div([], id='rebound-graph', style={'margin': '5px'}),
                                                html.Div([], id='rebound-histogram',  style={'margin': '5px'})
                                                
                                        ], style={'width':'100%', 'display': 'flex', 'margin': '20px'})
                                ])

@app.callback(  [Output(component_id='top10-table', component_property='children'),
                Output(component_id='rebound-graph', component_property='children'),
                Output(component_id='rebound-histogram', component_property='children')],
                [Input(component_id='input-season', component_property='value'),
                Input(component_id='input-ptype', component_property='value')])
def get_children(season, player_type):
    
    #get dataframe for top 10 table
    rebounds_avg_df = get_rebounds_df(season, player_type)
    top10_df = rebounds_avg_df.head(10)

    #get histogram
    hist = get_rebounds_hist(rebounds_avg_df)

    #get line or bar chart depending on selection
    if season is not None:
        figure = get_rebounds_bar(season, player_type)
    else:
        figure = get_rebounds_line(player_type)

    return [dash_table.DataTable(top10_df.to_dict(orient='records')), dcc.Graph(figure=figure), dcc.Graph(figure=hist)]

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
